<a href="https://colab.research.google.com/github/perfectpanda-works/machine-learning/blob/master/LEARNING_PYTORCH_WITH_EXAMPLES6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#nnモジュールのカスタマイズ

既存のモジュールのシーケンスよりも複雑なモデルを指定したい場合があります。

これらの場合、nn.Moduleをサブクラス化し、入力テンソルを受け取り、他のモジュールまたはTensorの他のautograd操作を使用して出力テンソルを生成する転送を定義することにより、独自のモジュールを定義できます。

これで、チュートリアルなどでよく見るクラスでのネットワークの定義の形になります。

In [1]:
# -*- coding: utf-8 -*-
import torch
device = torch.device("cuda:0") # Uncomment this to run on GPU
print(device)

cuda:0


In [4]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        コンストラクタ
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        順伝播の定義。xが入力するテンソル。
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred

In [5]:
# N  　：バッチサイズ
# D_in ：入力次元数
# H　　：隠れ層の次元数
# D_out：出力次元数
N, D_in, H, D_out = 64, 1000, 100, 10

# ランダムな入力データと出力データの作成
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# クラスとして定義したネットワークをインスタンス化する
model = TwoLayerNet(D_in, H, D_out)

# 誤差関数：平均二乗誤差
criterion = torch.nn.MSELoss(reduction='sum')
# 最適化手法：勾配降下法
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for t in range(500):
    # ①順伝播
    y_pred = model(x)

    # ②誤差の計算
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # ③逆伝播をして、重みの更新を行う。
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 2.448106288909912
199 0.029703807085752487
299 0.0006881364388391376
399 2.110017703671474e-05
499 7.303991083063011e-07


#制御と重みの共有について

In [10]:
# -*- coding: utf-8 -*-
import random
import torch

class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        コンストラクタ：「middle_linear」を追加
        """
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        順伝播の「middle_layer」を0〜3回forループで繰り返します。
        「middle_layer」をループ回数使い回しています。
        
        順伝播を定義するときには、Pythonのループや制御文を利用することができます。
        
        ここでは、計算グラフを定義するときに同じモジュールを何度も再利用しても大丈夫なことを示しています。
        これはLuaTorch(昔のPyTorch)からの改善点で、昔は各モジュールは１回しか使えませんでした。
        """
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred

N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = DynamicNet(D_in, H, D_out)

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):
    # 順伝播
    y_pred = model(x)

    # 損失の計算
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 最適化と重みの更新
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 24.734758377075195
199 3.7124757766723633
299 4.8652119636535645
399 1.488918662071228
499 1.2936948537826538
